In [1]:
# from TL import preproc_TL 
%matplotlib inline 
import matplotlib.pyplot as plt
from prep_data import Dataset 
from torch.utils.data import DataLoader
from LSTM_Net import LSTM_Net
import torch.nn as nn 
import torch 
import numpy as np
import math 
import gc
import time 
from skimage.transform import resize
import sys
sys.path.append("../")
from util import * 
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# fed the seed for reproducibility 

In [2]:
torch.backends.cudnn.deterministic = True
random.seed(1)
np.random.seed(1)
torch.manual_seed(1)
torch.cuda.manual_seed(1)



## Global variable 

In [3]:
# data paths and parameters
data_path = '../LSTM_SEAM_submitted_v2/data/'
base_path = f"{data_path}/base_seam.npy" 
mon_path = f"{data_path}/mon_seam.npy" 
Mclean_path = f"{data_path}/mon_seam_c.npy"

parm = {
 'nt': 2500,
 'ng': 501,
 'ns': 60,
 'dt':  0.0024,
 'dg': 0.02500000037252903,
 'ds': 0.17499999701976776,
 'ot': 0.0,
 'og': 0.0,
 'os': 1.0}
 

dt = parm['dt']
mxoffset = 5 # km 


# Hyperparameters for training
batchsz = 64 
num_epochs = 300 
LR = 0.002
hsz = 100
act = 'tanh'
n_layer=2
bias= False
dropout = 0.0


# # define the overburden window 
start = 550
last =  900 

# feature_length = 21
feature_length = 41

epoch=295
epoch=299
# hyperpar = f"_lr{LR}_nlayer{n_layer}_act{act}_bias{bias}_batchsz{batchsz}_time_start{start}_end{last}_mxoffset{mxoffset}"
hyperpar = f"_lr{LR}_nlayer{n_layer}_act{act}_bias{bias}_batchsz{batchsz}-featurelength{feature_length}_time_start{start}_end{last}_mxoffset{mxoffset}"

net_path = "./Network/"
net = f'{net_path}LSTM_SEAM_epc{epoch}{hyperpar}.pth'


last =  1600 

batchsz = 1 

# -----------------------------------------------#
# RSF PATH 
# path = '/home/alaliaa/Time_lapse_ML/ML_Models2/data/'
# base = path + 'Base_data.rsf'
# mon = path + 'MonNoisy_data.rsf'

In [4]:

base = np.load(base_path)
mon = np.load(mon_path)
Mclean = np.load(Mclean_path)


ns,ng,nt = base.shape
print ('ns',ns,'ng',ng,'nt',nt)


# Take only few shots 
# base = base[20:41:4,]
# mon = mon[20:41:4,]
# Mclean = Mclean[20:41:4,]
# ns,ng,nt_ = base.shape

base = base.reshape((ns*ng,nt))
mon = mon.reshape((ns*ng,nt))
Mclean = Mclean.reshape((ns*ng,nt))

print('data shapeed to 2d',base.shape)






ns 60 ng 501 nt 2500
data shapeed to 2d (30060, 2500)


In [5]:
''' Pre Processing the data for trainign 
    - specify the infer time window 
    - scale the data 
    - Rolling window ??! 

'''


def scale_data(data=None):
    print('shape of data that will be scaled', data.shape)
    assert len(data.shape)==2
    ng,nt = data.shape  
    
    data = data.T
    scaler= MinMaxScaler(feature_range=(-1,1))
    # scaler= StandardScaler()
    scaler.fit(data)
    scaled_data = scaler.transform(data)
    scaled_data = scaled_data.T 


    return scaled_data,scaler
    return scaled_data,scaler


def min_max_fit_custom(seq,a,b,minx,maxx): 

    seq = (b-a) * (seq-minx)/(maxx-minx)  + a 
    return seq       


import pickle 

with open('./minmax.pck','rb') as f:
    minmax = pickle.load(f)

# The training part from the trace 
base = base[:,start:last]
mon = mon[:,start:last] 
Mclean = Mclean[:,start:last] 


# Scaling 
# base[~np.all(base==0 , axis=1)], base_scaler = scale_data(base[~np.all(base==0 , axis=1)]) 
# mon[~np.all(mon==0 , axis=1)] , mon_scaler   = scale_data(mon[~np.all(mon==0 , axis=1)]) 
base = min_max_fit_custom(base,-1,1,minmax['min_in'],minmax['max_in'])
mon  = min_max_fit_custom(mon,-1,1,minmax['min_op'],minmax['max_op'])

# Scaling 
# base, base_scaler = scale_data(base) 
# mon , mon_scaler  = scale_data(mon) 

# move to torch 
base = torch.from_numpy(base).double()
mon  = torch.from_numpy(mon).double()





In [6]:
def window (x,dimension=-1,size=11,step=1):
    ''' x has to be pytorch tensor'''
    return x.unfold(dimension,size,step)


print(base.shape)
base_win =window(base,size=feature_length,step=1)
mon_win =window(mon,size=feature_length,step=1)[:,:,feature_length//2]
mon_win = mon_win.view(mon_win.shape[0],mon_win.shape[1],-1)


print(base_win.shape),print(mon_win.shape)


torch.Size([30060, 1050])
torch.Size([30060, 1010, 41])
torch.Size([30060, 1010, 1])


(None, None)

In [7]:
# Load the mdoel 

model = LSTM_Net(input_size=base_win.shape[-1], hidden_layer_size=hsz, output_size=mon_win.shape[-1],
                batch_sz=batchsz,num_lstm_layer=n_layer,activation=act,dropout=0,bias=bias)
# model = LSTM_Net2(1,hsz,1,batchsz)
model.load_state_dict(torch.load(net))
model.cuda().double()
model.eval()



LSTM_Net(
  (lstm): LSTM(41, 100, num_layers=2, bias=False, batch_first=True)
  (linear): Linear(in_features=100, out_features=1, bias=False)
)

In [8]:
''' Main inference ''' 

Pred = torch.zeros_like(mon_win)
count = 0 
# with torch.no_grad():
#     for i in range (base_win.shape[0]//batchsz): 
#         model.h_init()
#         inp = base_win[count:count+batchsz,]
#         zero_traces = (inp.numpy() != 0).argmax(axis = 1)
#         Pred_batch = model(inp[~zero_traces[0]].cuda())
#         Pred[count:count+batchsz,] = Pred_batch.detach().cpu()
#         count += batchsz
        
        
for i in range(base_win.shape[0]):
        inp = base_win[i,]
        if np.any(inp.numpy() != 0):
            Pred_batch = model(inp.reshape((1,inp.shape[0],inp.shape[1])).cuda())
            Pred[i:i+1,] = Pred_batch.detach().cpu()
        else: 
            Pred[i:i+1,]= torch.zeros_like(Pred[i:i+1,],dtype=torch.double)
        
# Pred = Pred.numpy()[:,:,0]
# mon = mon.detach().numpy()[:,:,0]
# base = base.detach().numpy()[:,:,0]


Pred = Pred[:,:,-1].numpy()
base = base.numpy()
mon = mon.numpy()

# Pred[~np.all(mon==0 , axis=1)] = mon_scaler.inverse_transform(Pred[~np.all(mon==0 , axis=1)].T).T    
# base[~np.all(base==0 , axis=1)] = base_scaler.inverse_transform(base[~np.all(base==0 , axis=1)].T).T    
# mon[~np.all(mon==0 , axis=1)] = mon_scaler.inverse_transform(mon[~np.all(mon==0 , axis=1)].T).T    


In [9]:
def min_max_inv_custom(seq,a,b,minx,maxx): 
    seq =(seq-a) * (maxx-minx) /(b-a) + minx
    return seq 

Pred = min_max_inv_custom(Pred,-1,1,minmax['min_op'],minmax['max_op'])

base = min_max_inv_custom(base,-1,1,minmax['min_in'],minmax['max_in'])
mon = min_max_inv_custom(mon,-1,1,minmax['min_op'],minmax['max_op'])

In [10]:
B=base.reshape((ns,ng,last-start))[:,:,feature_length//2:-(feature_length//2)]
M=mon.reshape((ns,ng,last-start))[:,:,feature_length//2:-(feature_length//2)]
Mc = Mclean.reshape((ns,ng,last-start))[:,:,feature_length//2:-(feature_length//2)]
Pr = Pred.reshape((ns,ng,last-start-feature_length+1))



# base= B 
# mon = M
# Mclean = Mc
# Pred = Pr
 
# base = base[1*ng:2*ng,:]
# mon = mon[1*ng:2*ng,:]
# Mclean = Mclean[1*ng:2*ng,:]
# Pred = Pred[1*ng:2*ng,:]

In [11]:
# Metrics 

def RMS(a):
    N = a.shape[0]
    return np.sqrt(1/N * np.sum(a**2))
#     return np.sqrt(np.mean(a**2))

def NRMS (B,M,ti,tf):
    N = B.shape[0]
    b = B[:,ti:tf].copy()
    m = M[:,ti:tf].copy()
    num = RMS(b-m)
    den = RMS(b)+RMS(m)
    score = 200 * num/den
    return score
    
def phi(b,m):
    num = np.sum(b*m)
    den = np.sqrt(np.sum(b**2)*np.sum(m**2))
    return num/den
    
def predictability(B,M,ti,tf):
    N = B.shape[0]
    b = B[:,ti:tf].copy()
    m = M[:,ti:tf].copy()
    w = b.shape[-1]
    tao = 0
    num = 0 
    den = 0
    for tao in range(w//2): 
        phi_bm = phi(b[tao:tao+w//2],m[tao:tao+w//2])
        phi_bb = phi(b[tao:tao+w//2],b[tao:tao+w//2])
        phi_mm = phi(m[tao:tao+w//2],m[tao:tao+w//2])
        num   += phi_bm * phi_bm
        den   += phi_bb * phi_mm
    score = 100 * num/den
    return score


In [12]:
# ''' Shot differeces plots ''' 
# import matplotlib
# from mpl_toolkits.axes_grid1 import make_axes_locatable


# # Shot plotting 

# # matplotlib.rc('image', cmap='gray')    
# matplotlib.rc('image', cmap='seismic') 
# font = {
# 'weight' : 'bold',
# 'size'   : 12}
# matplotlib.rc('font', **font)
# matplotlib.rcParams.update({'font.size': 12})
# vmax = 4e-6
# vmin = -vmax
# # vmax = .15
# # vmin = -.15
# # vmin, vmax = np.percentile((Mclean-base).T,[2,98])

# def plot_differences(shot1,shot2,shot3,ncols,savedir,metric=True):

#     # Measure metric window
#     ti = 0
#     tf = 400
#     print(f"metric window is [{start*dt}-{(start+tf)*dt}]")
#     fig, ax = plt.subplots(1,ncols,figsize=(16,5))
#     for i in range (ncols):
#         if metric:
#             nrms = NRMS(shot1[i],shot2[i],ti,tf)
#             pred = predictability(shot1[i],shot2[i],ti,tf)
#         shot_diff = shot1[i] - shot2[i]
#         im = ax[i].imshow(shot_diff[~np.all(shot3[i]==0 , axis=1)].T,
#             vmin=vmin,vmax=vmax,extent=[0,2000,last*dt,start*dt])
#         ax[i].axis("tight")
#         ax[i].set_ylabel("Time(s)",fontsize=16,fontweight='bold')
#         ax[i].axes.xaxis.set_visible(False)
#         if metric: 
#             ax[i].text(0.05,1.25,
#                 f"NRMS {nrms:.2f} \nPred {pred:.2f}",size=16)

#         divider = make_axes_locatable(ax[i])
#         cax = divider.append_axes('right', size='5%', pad=0.05)
#         fig.colorbar(im, cax=cax, orientation='vertical')
#     fig.tight_layout(pad=1.0)
#     fig.savefig(savedir, bbox_inches='tight')

  
    
    
# # plot_differences(M,Pr-Pr.mean(axis=2, keepdims=True),
# #                 ncols=5,savedir=f'./Fig/pred_diff{hyperpar}.png')
    
# plot_differences(M,Pr,Mc,
#                 ncols=5,savedir=f'./Fig/pred_diff{hyperpar}.png')

# plot_differences(M, B,Mc,
#                 ncols=5,savedir=f'./Fig/before_diff{hyperpar}.png')

# plot_differences(Mc,B,Mc,
#                 ncols=5,savedir=f'./Fig/target_diff{hyperpar}.png')

# plot_differences(Pr-Pr.mean(axis=2, keepdims=True),
#                  np.zeros_like(Pr), Mc,
#                 ncols=5,savedir=f'./Fig/pred_mon{hyperpar}.png',metric=False)

# plot_differences(M, np.zeros_like(M),Mc,
#                 ncols=5,savedir=f'./Fig/mon{hyperpar}.png',metric=False)

    

In [13]:
pred_diff = M -Pr
# pred_diff = M -(Pr-Pr.mean(axis=2, keepdims=True))
before_diff = M-B
true_diff = Mc- B

# Post processing step for unwanted behavior resulted from the scaling
# pred_diff[np.sum(np.abs(pred_diff),axis=2)> 0.1] = 0



In [14]:
path = './output_data/'

nt_resample = 7500
# ---------  pred difference 
pred_diff_all = np.zeros((ns,ng,nt))
pred_diff_all[:,:,start+(feature_length//2):last-(feature_length//2)] = pred_diff 
# Resamble to 7500 
pred_diff_all= resize(pred_diff_all,(pred_diff_all.shape[0],pred_diff_all.shape[1],nt_resample))    
filename = 'pred_diff.rsf@'
filename = path+filename
pred_diff_all.astype('float32').tofile(filename)
print("pred_diff shots is saved ")

# ---------  pred before diff 
before_diff_all = np.zeros((ns,ng,nt))
before_diff_all[:,:,start+(feature_length//2):last-(feature_length//2)] = before_diff 
# Resamble to 5000 
before_diff_all= resize(before_diff_all,(before_diff_all.shape[0],before_diff_all.shape[1],nt_resample))    
filename = 'before_diff.rsf@'
filename = path+filename
before_diff_all.astype('float32').tofile(filename)
print("before_diff shots is saved ")


# ---------  true before diff 
true_diff_all = np.zeros((ns,ng,nt))
true_diff_all[:,:,start+(feature_length//2):last-(feature_length//2)] = true_diff
# Resamble to 5000 
true_diff_all= resize(true_diff_all,(true_diff_all.shape[0],true_diff_all.shape[1],nt_resample))    
filename = 'true_diff.rsf@'
filename = path+filename
true_diff_all.astype('float32').tofile(filename)
print("true_diff shots is saved ")




pred_diff shots is saved 
before_diff shots is saved 
true_diff shots is saved 


In [15]:
nt

2500